In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### Amihud

In [7]:
query = '''
    SELECT DISTINCT
        TrdExctnDt,
        PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY Amihud) OVER (PARTITION BY TrdExctnDt) AS MedianAmihud
    FROM (
        SELECT
            CusipId,
            TrdExctnDt,
            ABS(RptdPr - LagRptdPr) / Volume AS Amihud
        FROM (    
            SELECT
                A.CusipId,
                A.TrdExctnDt,
                A.RptdPr,
                LAG(A.RptdPr) OVER (PARTITION BY A.CusipId ORDER BY A.TrdExctnDt) AS LagRptdPr,
                B.Volume
            FROM
                Trace_filteredWithRatings A
            INNER JOIN (    
                SELECT
                    CusipId,
                    TrdExctnDt,
                    MAX(TrdExctnTm) AS CloseTime,
                    SUM(EntrdVolQt) AS Volume
                FROM
                    Trace_filteredWithRatings
                GROUP BY
                    CusipId,
                    TrdExctnDt
            ) B ON A.CusipId = B.CusipId AND A.TrdExctnDt = B.TrdExctnDt AND A.TrdExctnTm = B.CloseTime
        ) C
    ) D
'''

# read sql
df = pd.read_sql(query, connection)
df.to_csv('source/amihud.csv', index=False)